# Longitudinal error rate fit GP

In [ ]:
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

### 1. Import the data

From the `dependencies` folder

In [ ]:
dependencies_folder = pathlib.Path().resolve().parent.joinpath("dependencies")

VelAcc = pd.read_csv(dependencies_folder.joinpath("VelAcc.csv"))

x_data = np.vstack((VelAcc["accelerations"].to_numpy(), VelAcc["velocities"].to_numpy())).T
y_data = VelAcc["long_dt_errors"].to_numpy()

### 2. Fit the GP on VelAcc

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Ignore the GPU

### Import the `lib` directory
import pathlib
import sys

repo_directory = pathlib.Path().resolve().parents[1]
lib_module_dir = str(repo_directory.joinpath("lib"))
if lib_module_dir not in sys.path:
    sys.path.insert(0, str(repo_directory.joinpath("lib")))

from dual_gp_model_SVGP import DualGaussianProcessWrapper, make_train_test_split

In [ ]:
EPOCHS: int = 500  # 2000
INDUCING_POINTS: int = 10
GRID_SIZE: int = 1000
MODEL_INFO: dict[str, str] = {"error": "longitudinal error rate", "param1": "acceleration", "param2": "velocity"}

In [ ]:
### Generate the model

GP_model_main = DualGaussianProcessWrapper(
    x_data=x_data,
    y_data=y_data,
    train_mask=make_train_test_split(len(y_data)),
    no_inducing_points=INDUCING_POINTS,
    data_directory=str(pathlib.Path().resolve()),  # current directory
)

In [ ]:
print(f"Going to train for {EPOCHS} epochs...")
GP_model_main.train(epochs=EPOCHS)
print(f"Plotting the posterior...")
GP_model_main.plot_posterior()
print(f"Generating the error model (interpolation grid)...")
GP_model_main.generate_error_model(grid_size=GRID_SIZE, model_info=MODEL_INFO)
print(f"Saving the error model...")
GP_model_main.save_error_model()
print("Done")

In [ ]:
### Save the GP picture
from matplotlib.legend_handler import HandlerTuple

fig = GP_model_main.plot_posterior(return_fig=True)
ax1, ax2, cbar = fig.get_axes()
ax2.remove()
cbar.remove()
ax1.set_xlabel("acceleration [m/s" + r"$^{2}$" + "]")
ax1.set_ylabel("velocity [m/s]")
ax1.set_zlabel("error [m/s]")

fig.suptitle("")
ax1.set_title("GP longitudinal trajectory following error rate")

ax1.set_box_aspect(aspect=None, zoom=0.88)

# ax1.legend([ax1.get_children()[1], ax1.get_children()[2]], ["mean", "variance"])

steps = 4
viridis_cm = mpl.cm.get_cmap("viridis")
colourmap_handle = []
for i in range(steps):
    colourmap_handle.append(mpl.patches.Patch(facecolor=viridis_cm(i / (steps - 1))))

ax1.legend(
    [colourmap_handle, ax1.get_children()[2], ax1.get_children()[0]],
    ["mean", "variance", "data"],
    handler_map={list: HandlerTuple(None, pad=0)},
)

fig.set_size_inches(10, 10)
fig.tight_layout()
fig.savefig("longitudinal_error_rate_gp.pdf", bbox_inches="tight")
fig.savefig("longitudinal_error_rate_gp.png", dpi=300, bbox_inches="tight")

### 3. Import the data for the scale factor GP

In [ ]:
VelCurv = pd.read_csv(dependencies_folder.joinpath("VelCurv.csv"))

x_data = VelCurv["curvatures"].to_numpy()
y_data = VelCurv["long_dt_errors"].to_numpy()

### 4. Fit the scale factor GP on VelCurv

In [ ]:
EPOCHS: int = 500  # 2000
INDUCING_POINTS: int = 10
GRID_SIZE: int = 1000
MODEL_INFO: dict[str, str] = {"error": "longitudinal error rate standard deviation scale", "param1": "curvature"}

In [ ]:
### Generate the model

GP_model_scale = DualGaussianProcessWrapper(
    x_data=x_data,
    y_data=y_data,
    train_mask=make_train_test_split(len(y_data)),
    no_inducing_points=INDUCING_POINTS,
    data_directory=str(pathlib.Path().resolve()),  # current directory
)

In [ ]:
print(f"Going to train for {EPOCHS} epochs...")
GP_model_scale.train(epochs=EPOCHS)
print(f"Plotting the posterior...")
GP_model_scale.plot_posterior()
print(f"Generating the error model (interpolation grid)...")
GP_model_scale.generate_error_model(grid_size=GRID_SIZE, model_info=MODEL_INFO)
print(f"Saving the error model...")
GP_model_scale.save_error_model()
print("Done")

In [ ]:
fig = GP_model_scale.plot_posterior(return_fig=True)

ax = fig.get_axes()[0]

ax.set_xlabel("curvature [m" + r"$^{-1}$" + "]")
ax.set_ylabel("error [m/s]")
ax.set_title("Scale function\n")
ax.text(0.5, 1.03, "of the longitudinal trajectory following error rate", fontsize=10, ha="center", transform=ax.transAxes)

fig.savefig("longitudinal_error_rate_scale_function_gp.pdf", bbox_inches="tight")
fig.savefig("longitudinal_error_rate_scale_function_gp.png", dpi=300, bbox_inches="tight")

### [Extra] Generate the error model with the scale function

In [ ]:
from error_model import ErrorModelWithStdScaleFunc

In [ ]:
def long_error_rate_std_scale_function(curvature: float) -> float:
    """factor = std[curvature] / std[0]"""
    _, std0 = GP_model_scale(np.array([[0]], dtype=float)) # std at zero
    _, std1 = GP_model_scale(np.array([[curvature]], dtype=float)) # std at the current curvature
    return std1 / std0

long_error_rate_model = ErrorModelWithStdScaleFunc.from_error_model(
    base_model=GP_model_main.error_model,
    std_scale_func=long_error_rate_std_scale_function,
)

In [ ]:
### Get some longitudinal error rate values for a range of values 

accelerations = np.linspace(-0.2, 0.2, 10)
velocities = np.linspace(0.1, 0.5, 10)
curvature = 1  # highest value for the whole trajectory
std_margins = 2  # amount of standard deviations wanted as a margin for the error

long_error_rate_model(np.vstack((accelerations, velocities)).T, stds_margin=std_margins, curvature = curvature)